### Import and init GPU

In [1]:
from IPython.display import Audio

In [2]:
import sys
sys.path.append('../../../../')

In [3]:
import numpy as np
import math

%matplotlib inline
import matplotlib.pyplot as plt

import pickle
import argparse
import time
import itertools
from copy import deepcopy
import tensorflow as tf

from tuner import HyperparameterTuner
from tuner import MyTask

use_tpu = False
use_gpu = True

if use_tpu:
    from tensorflow.contrib import tpu
    from tensorflow.contrib.cluster_resolver import TPUClusterResolver

if use_gpu:
    import os
    
%load_ext autoreload
%autoreload 2

In [4]:
if use_gpu:
    os.environ['CUDA_VISIBLE_DEVICES'] = '2'
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

In [5]:
if use_tpu:
    tpu_cluster = TPUClusterResolver(tpu=[tpu_name]).get_master()
    sess = tf.Session(tpu_cluster)
    sess.run(tpu.initialize_system())
elif use_gpu:
    sess = tf.Session(config=config)
else:
    sess = tf.Session()

In [6]:
from resnet import Network

In [7]:
network = Network()

In [8]:
task_home = ''
if use_tpu:
    pass
#     task_home = 'gs://continual_learning/permMNIST_EWC/'
else:
    task_home = '../../../../../'

cur_dir = './'
checkpoint_path = cur_dir + 'checkpoints_0/'
summaries_path = cur_dir + 'summaries_0/'
data_path = task_home + 'cifar-100-python/'
split_path = './split.txt' 
if use_tpu:
    tpu_name = 'gectpu'
    
BATCH_SIZE = 128

### Dataset loading functions

In [9]:
label_smooth_param = 0

def splitDataset(dataset, dataset_split, seed):
    np.random.seed(seed)
    task_list = []
    train_labels = np.argmax(dataset.train.labels, axis=1)
    validation_labels = np.argmax(dataset.validation.labels, axis=1)
    test_labels = np.argmax(dataset.test.labels, axis=1)
    for i in range(len(dataset_split)):
        cur_train_indices = [False] * dataset.train.images.shape[0]
        cur_validation_indices = [False] * dataset.validation.images.shape[0]
        cur_test_indices = [False] * dataset.test.images.shape[0]
        for j in range(len(dataset_split[i])):
            cur_train_indices = np.logical_or(cur_train_indices, (train_labels == dataset_split[i][j]))
            cur_validation_indices = np.logical_or(cur_validation_indices, (validation_labels == dataset_split[i][j]))
            cur_test_indices = np.logical_or(cur_test_indices, (test_labels == dataset_split[i][j]))

        task = deepcopy(dataset)
        task.train.images = task.train.images[cur_train_indices]
        task.train.labels = task.train.labels[cur_train_indices]
        task.validation.images = task.validation.images[cur_validation_indices]
        task.validation.labels = task.validation.labels[cur_validation_indices]
        task.test.images = task.test.images[cur_test_indices]
        task.test.labels = task.test.labels[cur_test_indices]
        task = MyTask(task)
        task_list.append(task)

    return task_list
    
def smoothLabels(dataset):
    train_labels = dataset.train.labels
    train_labels_argmax = np.argmax(train_labels, axis=1)
    train_labels = train_labels + label_smooth_param / (train_labels.shape[1] - 1)
    train_labels[range(train_labels.shape[0]), train_labels_argmax] = 1 - label_smooth_param
    dataset.train._labels = train_labels

class TempDataset(object):
    def __init__(self):
        self.images = None
        self.labels = None
    
class TempTask(object):
    def __init__(self):
        self.train = TempDataset()
        self.validation = TempDataset()
        self.test = TempDataset()
    
    
def readDatasets():
    num_class = 100
    labels_list = list(range(num_class))
    seed = 0
    np.random.seed(seed)
    np.random.shuffle(labels_list)
    split = []
    task_weights = []
    
    split = [labels_list[ : 91]]
    task_weights = [0.91]
    for single_label in labels_list[91 : ]:
        split.append([single_label])
        task_weights.append(0.01)
    num_tasks = len(split)
    
    with open(data_path + 'train', 'rb') as f:
        f_train_data = pickle.load(f, encoding='bytes')
        
    with open(data_path + 'test', 'rb') as f:
        f_test_data = pickle.load(f, encoding='bytes')
        
    cifar_100 = TempTask()
    temp_train_labels = np.array(f_train_data[b'fine_labels'], dtype=np.int32)
    temp_test_labels = np.array(f_test_data[b'fine_labels'], dtype=np.int32)
    f_train_data[b'fine_labels'] = np.zeros((temp_train_labels.shape[0], num_class))
    (f_train_data[b'fine_labels'])[range(temp_train_labels.shape[0]), temp_train_labels] = 1
    f_test_data[b'fine_labels'] = np.zeros((temp_test_labels.shape[0], num_class))
    (f_test_data[b'fine_labels'])[range(temp_test_labels.shape[0]), temp_test_labels] = 1
    f_train_data[b'data'] = np.reshape(f_train_data[b'data'], (-1, 3, 32, 32))
    f_test_data[b'data'] = np.reshape(f_test_data[b'data'], (-1, 3, 32, 32))
    f_train_data[b'data'] = np.transpose(f_train_data[b'data'], (0, 2, 3, 1))
    f_test_data[b'data'] = np.transpose(f_test_data[b'data'], (0, 2, 3, 1))
    
    tr_data = f_train_data[b'data']
    te_data = f_test_data[b'data']
    # normalizing data
    avg = np.mean(tr_data, axis=(0, 1, 2))
    std = np.std(tr_data, axis=(0, 1, 2))
    
    f_train_data[b'data'] = (tr_data - avg) / std
    f_test_data[b'data'] = (te_data - avg) / std
    
    seed = 0
    np.random.seed(0)
    shuffle_train_perm = np.random.permutation(f_train_data[b'data'].shape[0])
    f_train_data[b'data'] = f_train_data[b'data'][shuffle_train_perm]
    f_train_data[b'fine_labels'] = f_train_data[b'fine_labels'][shuffle_train_perm]
    
    num_val_per_class = 20
    
    for i in range(num_class):
        pos = (np.argmax(f_train_data[b'fine_labels'], axis=1) == i)
        
        if (i == 0):
            cifar_100.validation.images = (f_train_data[b'data'][pos])[0 : num_val_per_class]
            cifar_100.validation.labels = (f_train_data[b'fine_labels'][pos])[0 : num_val_per_class]

            cifar_100.train.images = (f_train_data[b'data'][pos])[num_val_per_class : ]
            cifar_100.train.labels = (f_train_data[b'fine_labels'][pos])[num_val_per_class : ]
        else:
            cifar_100.validation.images = np.concatenate((cifar_100.validation.images, (f_train_data[b'data'][pos])[0 : num_val_per_class]))
            cifar_100.validation.labels = np.concatenate((cifar_100.validation.labels, (f_train_data[b'fine_labels'][pos])[0 : num_val_per_class]))

            cifar_100.train.images = np.concatenate((cifar_100.train.images, (f_train_data[b'data'][pos])[num_val_per_class : ]))
            cifar_100.train.labels = np.concatenate((cifar_100.train.labels, (f_train_data[b'fine_labels'][pos])[num_val_per_class : ]))
        
    cifar_100.test.images = f_test_data[b'data']
    cifar_100.test.labels = f_test_data[b'fine_labels']
    
    shuffle_train_perm = np.random.permutation(cifar_100.train.images.shape[0])
    cifar_100.train.images = cifar_100.train.images[shuffle_train_perm]
    cifar_100.train.labels = cifar_100.train.labels[shuffle_train_perm]
    
    if (label_smooth_param != 0):
        smoothLabels(cifar_100)
        
    task_list = splitDataset(cifar_100, split, seed)
    return split, num_tasks, task_weights, task_list

### Create tuner object and train!!

In [10]:
input_shape = (32, 32, 3)
output_shape = (100, )

In [11]:
tuner = HyperparameterTuner(sess=sess, network=network, 
                            input_shape=input_shape, output_shape=output_shape,
                            checkpoint_path=checkpoint_path, summaries_path=summaries_path, 
                            readDatasets=readDatasets, load_best_hparams=False)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [12]:
tuner.setPerExampleAppend(1.0)
tuner.updateTunerHparams({'mask_softmax' : True})

### Training each task separately

In [ ]:
t = 0
learning_rates = [1e-1]
momentums = [0.9]
regs = [0.0001]
dropout_input_probs = [1.0]
dropout_hidden_probs = [0.9]
prod = list(itertools.product(regs, dropout_input_probs, dropout_hidden_probs, momentums, learning_rates))
hparams = []
for hparams_tuple in prod:
    cur_dict = {}
    cur_dict['reg'] = hparams_tuple[0]
    cur_dict['dropout_input_prob'] = hparams_tuple[2]
    cur_dict['dropout_hidden_prob'] = hparams_tuple[2]
    cur_dict['momentum'] = hparams_tuple[3]
    cur_dict['learning_rate'] = hparams_tuple[4]
    hparams.append(cur_dict)

tuner.hparams_list[t] = hparams

In [ ]:
num_epochs = 160
num_updates = math.ceil(tuner.task_list[t].train.images.shape[0] / BATCH_SIZE) * num_epochs
tuner.print_every = 10

In [ ]:
start_time = time.time()
best_avg, best_hparams = tuner.tuneOnTask(t, BATCH_SIZE, 
                                          save_weights=False, 
                                          num_updates=num_updates, verbose=True, 
                                          random_crop_flip=True)
print("time taken : %d" % (time.time() - start_time))
sound_file = '/mnt/a99/d0/shriramsb/code/Alan Walker - Alone.mp3'
Audio(sound_file, autoplay=True)

In [ ]:
import math
plt.rcParams['figure.figsize'] = [10, 5]
lr_scatter = ([math.log10(h['learning_rate']) for h in hparams])
dropout_scatter = [h['dropout_hidden_prob'] for h in hparams]
colors = []
for i in range(len(hparams)):
    cur_hparam_tuple = tuner.hparamsDictToTuple(hparams[i], tuner.tuner_hparams)
    colors.append(tuner.results_list[t][cur_hparam_tuple]['best_avg'])
    
marker_size = 100
fig, ax = plt.subplots()
plt.scatter(lr_scatter, dropout_scatter, marker_size, c=colors, edgecolors='black')
plt.colorbar()
for i in range(len(lr_scatter)):
    ax.annotate(str('%0.4f' % (colors[i], )), (lr_scatter[i], dropout_scatter[i]))
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]
best_hparams_tuple = tuner.hparamsDictToTuple(hparams[0], tuner.tuner_hparams)
cur_res = tuner.results_list[t][best_hparams_tuple]
x = np.arange(0, cur_res['total_updates'], cur_res['updates_per_epoch'])
cur_best_avg = cur_res['best_avg']
cur_best_avg_updates = cur_res['best_avg_updates']
updates_per_epoch = cur_res['updates_per_epoch']
# print("dropout: %f, fisher_multiplier: %e, lr: %e" % (k[0], k[1], k[2]))
print("cur_best_avg: %e, num_updates: %d" % (cur_best_avg, cur_best_avg_updates))
print("best val_acc: %s" % (str(np.array(cur_res['val_acc'])[:, (cur_best_avg_updates // updates_per_epoch - 1) // tuner.eval_frequency])))
# plt.plot(cur_res['loss_with_penalty'], color='g')
plt.plot(cur_res['loss'], color='m')
plt.plot(x, cur_res['val_loss'][-1], color='b')
plt.show()
# plt.ylim(ymin=0.9)
plt.plot(cur_res['val_acc'][0], color='b', )
# plt.plot(cur_res['val_acc'][1], color='g')

plt.show()

In [ ]:
# print(best_avg, best_hparams)
VALIDATION_BATCH_SIZE = 128
print(tuner.validationAccuracy(t, VALIDATION_BATCH_SIZE, restore_model=False))

In [ ]:
t = 1
learning_rates = [1e-1]
momentums = [0.9]
regs = [0.0001]
dropout_input_probs = [1.0]
dropout_hidden_probs = [0.9]
prod = list(itertools.product(regs, dropout_input_probs, dropout_hidden_probs, momentums, learning_rates))
hparams = []
for hparams_tuple in prod:
    cur_dict = {}
    cur_dict['reg'] = hparams_tuple[0]
    cur_dict['dropout_input_prob'] = hparams_tuple[2]
    cur_dict['dropout_hidden_prob'] = hparams_tuple[2]
    cur_dict['momentum'] = hparams_tuple[3]
    cur_dict['learning_rate'] = hparams_tuple[4]
    hparams.append(cur_dict)

tuner.hparams_list[t] = hparams

In [ ]:
num_epochs = 160
num_updates = math.ceil(tuner.task_list[t].train.images.shape[0] / BATCH_SIZE) * num_epochs
tuner.print_every = 10

In [ ]:
start_time = time.time()
best_avg, best_hparams = tuner.tuneOnTask(t, BATCH_SIZE, 
                                          save_weights=False, 
                                          num_updates=num_updates, verbose=True, 
                                          random_crop_flip=True)
print("time taken : %d" % (time.time() - start_time))
sound_file = '/mnt/a99/d0/shriramsb/code/Alan Walker - Alone.mp3'
Audio(sound_file, autoplay=True)

In [ ]:
import math
plt.rcParams['figure.figsize'] = [10, 5]
lr_scatter = ([math.log10(h['learning_rate']) for h in hparams])
dropout_scatter = [h['dropout_hidden_prob'] for h in hparams]
colors = []
for i in range(len(hparams)):
    cur_hparam_tuple = tuner.hparamsDictToTuple(hparams[i], tuner.tuner_hparams)
    colors.append(tuner.results_list[t][cur_hparam_tuple]['best_avg'])
    
marker_size = 100
fig, ax = plt.subplots()
plt.scatter(lr_scatter, dropout_scatter, marker_size, c=colors, edgecolors='black')
plt.colorbar()
for i in range(len(lr_scatter)):
    ax.annotate(str('%0.4f' % (colors[i], )), (lr_scatter[i], dropout_scatter[i]))
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]
best_hparams_tuple = tuner.hparamsDictToTuple(hparams[0], tuner.tuner_hparams)
cur_res = tuner.results_list[t][best_hparams_tuple]
x = np.arange(0, cur_res['total_updates'], cur_res['updates_per_epoch'])
cur_best_avg = cur_res['best_avg']
cur_best_avg_updates = cur_res['best_avg_updates']
updates_per_epoch = cur_res['updates_per_epoch']
# print("dropout: %f, fisher_multiplier: %e, lr: %e" % (k[0], k[1], k[2]))
print("cur_best_avg: %e, num_updates: %d" % (cur_best_avg, cur_best_avg_updates))
print("best val_acc: %s" % (str(np.array(cur_res['val_acc'])[:, (cur_best_avg_updates // updates_per_epoch - 1) // tuner.eval_frequency])))
# plt.plot(cur_res['loss_with_penalty'], color='g')
plt.plot(cur_res['loss'], color='m')
plt.plot(x, cur_res['val_loss'][-1], color='b')
plt.show()
# plt.ylim(ymin=0.9)
plt.plot(cur_res['val_acc'][0], color='b', )
plt.plot(cur_res['val_acc'][1], color='g')

plt.show()

### Train tasks together

In [13]:
t = 0
learning_rates = [(((49, 1e-1), (63, 1e-1 / 5), 1e-1 / (5 * 5)), (1e-1, ))]
momentums = [0.9]
regs = [0.00001]
dropout_input_probs = [1.0]
dropout_hidden_probs = [0.9]
prod = list(itertools.product(regs, dropout_input_probs, dropout_hidden_probs, momentums, learning_rates))
hparams = []
for hparams_tuple in prod:
    cur_dict = {}
    cur_dict['reg'] = hparams_tuple[0]
    cur_dict['dropout_input_prob'] = hparams_tuple[2]
    cur_dict['dropout_hidden_prob'] = hparams_tuple[2]
    cur_dict['momentum'] = hparams_tuple[3]
    cur_dict['learning_rate'] = hparams_tuple[4]
    hparams.append(cur_dict)
    
for i in range(0, t + 1):
    tuner.hparams_list[i] = hparams
    

In [14]:
num_hparams = len(hparams)
num_epochs = 70
num_updates = math.ceil(tuner.task_list[t].train.images.shape[0] / BATCH_SIZE) * num_epochs
tuner.print_every = 100

In [15]:
best_avg, best_hparams_index = tuner.tuneTasksInRange(0, t, BATCH_SIZE, num_hparams, 
                                                        num_updates=num_updates, verbose=True, 
                                                        random_crop_flip=True, 
                                                        equal_weights=True)

Training with T=None,alpha=0.0,dropout_hidden_prob=0.9,dropout_input_prob=0.9,epsilon=0.0,fisher_multiplier=0.0,learning_rate=too_long,momentum=0.9,reg=1e-05,bf_num_images=2000,mask_softmax=True,old:new=1.0,task=0
epoch: 0, iter: 100/342, loss: 3.952404, accuracy: 0.054688
epoch: 0, iter: 200/342, loss: 3.717833, accuracy: 0.140625
epoch: 0, iter: 300/342, loss: 3.579928, accuracy: 0.132812
epoch: 1, iter: 0/342, validation accuracies: [0.11593407], average train loss: 3.997124, average train accuracy: 0.073077
epoch: 1, iter: 58/342, loss: 3.508539, accuracy: 0.171875
epoch: 1, iter: 158/342, loss: 3.405050, accuracy: 0.179688
epoch: 1, iter: 258/342, loss: 3.413889, accuracy: 0.132812
epoch: 2, iter: 0/342, validation accuracies: [0.19175824], average train loss: 3.452638, average train accuracy: 0.158808
epoch: 2, iter: 16/342, loss: 3.160216, accuracy: 0.226562
epoch: 2, iter: 116/342, loss: 3.038945, accuracy: 0.234375
epoch: 2, iter: 216/342, loss: 3.098219, accuracy: 0.234375
ep

epoch: 24, iter: 292/342, loss: 0.866778, accuracy: 0.726562
epoch: 25, iter: 0/342, validation accuracies: [0.55879122], average train loss: 0.933357, average train accuracy: 0.718841
epoch: 25, iter: 50/342, loss: 0.901033, accuracy: 0.703125
epoch: 25, iter: 150/342, loss: 0.841786, accuracy: 0.703125
epoch: 25, iter: 250/342, loss: 1.177395, accuracy: 0.679688
epoch: 26, iter: 0/342, validation accuracies: [0.59230769], average train loss: 0.918690, average train accuracy: 0.723159
epoch: 26, iter: 8/342, loss: 0.766311, accuracy: 0.757812
epoch: 26, iter: 108/342, loss: 0.750028, accuracy: 0.765625
epoch: 26, iter: 208/342, loss: 0.886194, accuracy: 0.718750
epoch: 26, iter: 308/342, loss: 0.742012, accuracy: 0.765625
epoch: 27, iter: 0/342, validation accuracies: [0.58516483], average train loss: 0.884694, average train accuracy: 0.733530
epoch: 27, iter: 66/342, loss: 0.868346, accuracy: 0.750000
epoch: 27, iter: 166/342, loss: 0.922596, accuracy: 0.765625
epoch: 27, iter: 266/3

epoch: 49, iter: 142/342, loss: 0.383800, accuracy: 0.882812
epoch: 49, iter: 242/342, loss: 0.408480, accuracy: 0.875000
epoch: 50, iter: 0/342, validation accuracies: [0.6489011], average train loss: 0.420427, average train accuracy: 0.871276
epoch: 50, iter: 0/342, loss: 0.223567, accuracy: 0.921875
epoch: 50, iter: 100/342, loss: 0.508505, accuracy: 0.804688
epoch: 50, iter: 200/342, loss: 0.451786, accuracy: 0.875000
epoch: 50, iter: 300/342, loss: 0.288275, accuracy: 0.898438
epoch: 51, iter: 0/342, validation accuracies: [0.65329671], average train loss: 0.364282, average train accuracy: 0.887770
epoch: 51, iter: 58/342, loss: 0.395462, accuracy: 0.898438
epoch: 51, iter: 158/342, loss: 0.233014, accuracy: 0.945312
epoch: 51, iter: 258/342, loss: 0.344085, accuracy: 0.867188
epoch: 52, iter: 0/342, validation accuracies: [0.65384615], average train loss: 0.344877, average train accuracy: 0.894394
epoch: 52, iter: 16/342, loss: 0.338242, accuracy: 0.906250
epoch: 52, iter: 116/34

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]
best_hparams_tuple = tuner.hparamsDictToTuple(hparams[0], tuner.tuner_hparams)
cur_res = tuner.results_list[t][best_hparams_tuple][0]
x = np.arange(0, cur_res['total_updates'], cur_res['updates_per_epoch'])
cur_best_avg = cur_res['best_avg']
cur_best_epoch = cur_res['best_epoch']
updates_per_epoch = cur_res['updates_per_epoch']
# print("dropout: %f, fisher_multiplier: %e, lr: %e" % (k[0], k[1], k[2]))
print("cur_best_avg: %e, num_updates: %d" % (cur_best_avg, cur_best_epoch))
print("best val_acc: %s" % (str(np.array(cur_res['val_acc'])[:, (cur_best_epoch - 1) // tuner.eval_frequency])))
# plt.plot(cur_res['loss_with_penalty'], color='g')
plt.plot(cur_res['loss'], color='m')
plt.plot(x, cur_res['val_loss'][-1], color='b')
plt.show()
# plt.ylim(ymin=0.9)
plt.plot(cur_res['val_acc'][-1], color='b', )
plt.plot(cur_res['val_acc'][0], color='g', )
# plt.plot(cur_res['val_acc'][1], color='g')

plt.show()

In [16]:
t = 1
learning_rates = [(((49, 1e-1), (63, 1e-1 / 5), 1e-1 / (5 * 5)), (1e-1, ))]
momentums = [0.9]
regs = [0.00001]
dropout_input_probs = [1.0]
dropout_hidden_probs = [0.9]
prod = list(itertools.product(regs, dropout_input_probs, dropout_hidden_probs, momentums, learning_rates))
hparams = []
for hparams_tuple in prod:
    cur_dict = {}
    cur_dict['reg'] = hparams_tuple[0]
    cur_dict['dropout_input_prob'] = hparams_tuple[2]
    cur_dict['dropout_hidden_prob'] = hparams_tuple[2]
    cur_dict['momentum'] = hparams_tuple[3]
    cur_dict['learning_rate'] = hparams_tuple[4]
    hparams.append(cur_dict)
    
for i in range(1, t + 1):
    tuner.hparams_list[i] = hparams

In [17]:
num_hparams = len(hparams)
num_epochs = 70
num_updates = math.ceil(tuner.task_list[t].train.images.shape[0] / BATCH_SIZE) * num_epochs
tuner.print_every = 100

In [18]:
best_avg, best_hparams_index, test_acc = tuner.tuneTasksInRange(1, t, BATCH_SIZE, num_hparams, 
                                                        num_updates=num_updates, verbose=True, 
                                                        random_crop_flip=True, 
                                                        equal_weights=True, 
                                                        eval_test_dataset=True)

Training with T=None,alpha=0.0,dropout_hidden_prob=0.9,dropout_input_prob=0.9,epsilon=0.0,fisher_multiplier=0.0,learning_rate=too_long,momentum=0.9,reg=1e-05,bf_num_images=2000,mask_softmax=True,old:new=1.0,task=1
Restoring paramters from dropout_hidden_prob=0.9,dropout_input_prob=0.9,learning_rate=too_long,momentum=0.9,reg=1e-05,bf_num_images=2000,mask_softmax=True,old:new=1.0,task=0
INFO:tensorflow:Restoring parameters from ./checkpoints_0/dropout_hidden_prob=0.9,dropout_input_prob=0.9,learning_rate=too_long,momentum=0.9,reg=1e-05,bf_num_images=2000,mask_softmax=True,old:new=1.0,task=0.ckpt-23940
epoch: 1, iter: 0/4, validation accuracies: [0.21098901 0.        ], average train loss: 13.676358, average train accuracy: 0.298828
epoch: 2, iter: 0/4, validation accuracies: [0.1032967 0.       ], average train loss: 7.646547, average train accuracy: 0.292969
epoch: 3, iter: 0/4, validation accuracies: [0.06538462 0.        ], average train loss: 5.614647, average train accuracy: 0.193359

epoch: 58, iter: 0/4, validation accuracies: [0.47692308 0.40000001], average train loss: 1.641357, average train accuracy: 0.544922
epoch: 59, iter: 0/4, validation accuracies: [0.48186814 0.40000001], average train loss: 1.684681, average train accuracy: 0.525391
epoch: 60, iter: 0/4, validation accuracies: [0.48351649 0.40000001], average train loss: 1.584028, average train accuracy: 0.548828
epoch: 61, iter: 0/4, validation accuracies: [0.48186813 0.40000001], average train loss: 1.637903, average train accuracy: 0.560547
epoch: 62, iter: 0/4, validation accuracies: [0.48516483 0.34999999], average train loss: 1.684457, average train accuracy: 0.541016
epoch: 63, iter: 0/4, validation accuracies: [0.48461538 0.34999999], average train loss: 1.629075, average train accuracy: 0.558594
epoch: 64, iter: 0/4, validation accuracies: [0.48571428 0.40000001], average train loss: 1.617810, average train accuracy: 0.519531
epoch: 65, iter: 0/4, validation accuracies: [0.48571429 0.40000001],

In [19]:
print("test_acc:", test_acc)

test_acc: [[0.5118681318812318, 0.4099999964237213]]


In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]
best_hparams_tuple = tuner.hparamsDictToTuple(hparams[0], tuner.tuner_hparams)
cur_res = tuner.results_list[t][best_hparams_tuple][0]
x = np.arange(0, cur_res['total_updates'], cur_res['updates_per_epoch'])
cur_best_avg = cur_res['best_avg']
cur_best_epoch = cur_res['best_epoch']
updates_per_epoch = cur_res['updates_per_epoch']
# print("dropout: %f, fisher_multiplier: %e, lr: %e" % (k[0], k[1], k[2]))
print("cur_best_avg: %e, num_updates: %d" % (cur_best_avg, cur_best_epoch))
print("best val_acc: %s" % (str(np.array(cur_res['val_acc'])[:, (cur_best_epoch - 1) // tuner.eval_frequency])))
# plt.plot(cur_res['loss_with_penalty'], color='g')
# plt.plot(cur_res['loss'], color='m')
# plt.plot(x, cur_res['val_loss'][-1], color='b')
# plt.show()
# plt.ylim(ymin=0.9)
plt.plot(cur_res['val_acc'][-1], color='b', )
plt.plot(cur_res['val_acc'][0], color='g', )
# plt.plot(cur_res['val_acc'][1], color='g')

plt.show()

In [ ]:
def getAllCosineSimilarity(wts, init_classes, only_dot=False):
    num_init_class = len(init_classes)
    cosine_wts = [[0.0 for _ in range(num_init_class)] for _ in range(num_init_class)]
    for i in range(num_init_class):
        for j in range(num_init_class):
            w_i = wts[0][:, init_classes[i]]
            w_j = wts[0][:, init_classes[j]]
            if (only_dot):
                cosine_wts[i][j] = np.sum(w_i * w_j)
            else:
                cosine_wts[i][j] = np.sum(w_i * w_j) / np.sqrt(np.sum(w_i ** 2)) / np.sqrt(np.sum(w_j ** 2))
    return cosine_wts

In [ ]:
num_tasks_test = 10
wts = [None for _ in range(num_tasks_test)]
init_classes = tuner.split[0]
cosine_sim_wts = [None for _ in range(num_tasks_test)]
dot_sim_wts = [None for _ in range(num_tasks_test)]
for i in range(num_tasks_test):
    tuner.test(i, BATCH_SIZE, restore_model=True, hparams=tuner.hparams_list[i][0])
    wts[i] = sess.run([v for v in tf.all_variables() if 'dense' in v.name and 'kernel:0' in v.name])
    cosine_sim_wts[i] = np.array(getAllCosineSimilarity(wts[i], init_classes, only_dot=False))
    dot_sim_wts[i] = np.array(getAllCosineSimilarity(wts[i], init_classes, only_dot=True))
    

In [ ]:
for i in range(num_tasks_test):
#     plt.imshow(cosine_sim_wts[i])
    print("task", i)
    print("sum cosine: ", np.sum(cosine_sim_wts[i]) - num_init_class)
    print("sum dot: ", np.sum(dot_sim_wts[i]) - np.sum(wts[i][0][: , init_classes] ** 2))
    print("sum norms: ", np.sum(np.sqrt(np.sum(wts[i][0][:, init_classes] ** 2, axis=0))))
#     plt.show()

In [ ]:
tuner.saveResultsList()
tuner.saveBestHparams()

In [ ]:
TEST_BATCH_SIZE = 128
test_accuracies = []
for i in range(t + 1):
    accuracy = tuner.test(i, TEST_BATCH_SIZE, restore_model=False)
    test_accuracies.append(accuracy)

In [ ]:
for i in range(t + 1):
    print(test_accuracies[i])

In [ ]:
if use_tpu:
    sess.run(tpu.shutdown_system())

sess.close()

In [ ]:
import shelve
filename='code_state.bak'
my_shelf = shelve.open(filename,'n') # 'n' for new

for key in dir():
    try:
        my_shelf[key] = globals()[key]
    except TypeError:
        #
        # __builtins__, my_shelf, and imported modules can not be shelved.
        #
        print('ERROR shelving: {0}'.format(key))
my_shelf.close()